# Stokes coefficients estimation
In this set of notebooks, we show simulations of retrival of a set of real spherical harmonics gravity coefficients from a set of gravitational measurements (either potential or acceleration). This is a simplified example of what is performed in an orbit determination (OD) campaign, where the Stokes coefficients are estimated, along with other parameters, from a set of radio-tracking measurements of an orbiting spacecraft.

## **Case 4**: Fitting Bennu particle accelerations
Here we use synthetic accelerations computed for particles in orbit around asteroid Bennu using the polyhedron method. Therefore, here the synthetic data are not generated using spherical harmonics. We then compare the estimated Stokes coefficients with those corresponding to the polyhedral mesh

In [1]:
import os
from time import time

import numpy as np
import plotly.graph_objects as go
import pyshtools as sh
import matplotlib.pyplot as plt
import pickle as pk
import json

import scripts
from scripts._units import *

from IPython.display import IFrame
import plotly.io as pio
pio.templates.default = "plotly_white"

### Settings

In [2]:
out_name = "bennu_particles" # outputs destination folder (a subfolder of out)
out_path = os.path.join("out", out_name)
if not os.path.exists(out_path):
    os.makedirs(out_path)

use_potential = False # Whether to work with potential (flag to True) or with accelerations (flag to False)
perturb_msr = False # Whether to introduce additive Gaussian noise in the syntehtic measurements
compute_covariance = True # Additionally compute uncertainties for the estimated coefficients (~20% increase in inversion time)
check_acc_msr = False # Compare acceleartion measurements with those given by pyshtools

msr_dim = 1 if use_potential else 3 # dimensions of each measurerment (i.e. scalar or vector)
partials_func = scripts.pot_cnm_partials if use_potential else scripts.acc_cnm_partials # function to use for partials computation
val_lbl = "U_grav (m^2/s^2)" if use_potential else "|a_grav| (m/s^2)" # label for plots
rng = np.random.default_rng()

### Ground-truth gravity
In this case, the reference Stokes coefficients are computed for the polyhedral mesh representing the shape of asteroid Bennu, using our GILA tool and assuming a homogeneous density distribution of $1200 kg/m^3$. By tracking particles orbiting the asteroids (whose trajectories are used below), the OSIRIS-REx team obtained a SH gravity field up to degree 10. Therefore, here we choose to also estimate a set of Stokes coefficients with `n_max_est=10`.
The coefficients in `cnm_mat_sim` are not used to generate the synthetic measurements (which have been computed separately and are only loaded here). They only provide the ground truth.

In [3]:
# SH degree cutoff (for both simulation and estimation, and in any case limited by that of the file)
n_max_sim = 10 # macimum degree of the coefficients used to simulate the measurements
n_max_est = 10 # maximum degree of the coefficients to be estimated
n_max = max(n_max_sim, n_max_est)

# Read SH coefficients, and cut to n_max_sim
file_name = "spice/data/bennu/sh_homogeneous.json"
cnm_mat, r_0, gm = scripts.read_cnm_json(file_name)
cnm_mat_sim = cnm_mat[:, : n_max_sim + 1, : n_max_sim + 1]
n_max_sim = cnm_mat_sim.shape[-1]-1

# Index array and dictionary to go from a matrix of Cnm to a 1D array, and vice-versa
cnm_idx_sim, cnm_map_sim = scripts.get_cnm_idx(n_max_sim)
cnm_idx_est, cnm_map_est = scripts.get_cnm_idx(n_max_est)

cnm_mat_est = np.empty((2, n_max_est + 1, n_max_est + 1), dtype=bool)

### Measurements points
The locations of the measurements are obtained from the SPICE kernels of the tracked particles in orbit around Bennu, which are provided by the OSIRIS-REx team. These particles constitute rubble material escaping from the surface of the asteroid. Given their abundance and the low altitudes of their orbits, tracking these particles allowed to greatly improve the resolution of the estimated gravity, which aws around degree-4 using spacecraft radio-tracking alone. This specific example and irregular bodies in general are particularly challenging for the gravity representation via spherical harmonics, due to the very localized variations in the field and the divergence effects close to the surface.
In this simulation, given the orbits of selected particles and a polyhedral shape model for Bennu, we use the ESA polyhedral-gravity library to generate synthetic gravity accelerations at points along the trajectories, with a time-step of 300 seconds. Again, these measurements 

In [4]:
file_path = "spice/data/bennu/particles_grav.pkl"
with open(file_path, "rb") as f:
    state_mat, sph_coords, sim_msr_vec = pk.load(f)
n_msr_pts = sim_msr_vec.shape[0]
sim_msr_vec = sim_msr_vec.reshape(n_msr_pts * msr_dim)

## Forward model

### Computing partials
The partial computation and the filtering are performed in the same way as in the previous notebook

In [5]:
print("Computing Normal equations...")
ts = time()

# measurements standard deviation. Can be a vector of size n_msr or a float.
msr_noise = 1e-4 * gm / np.power(r_0, 1 if use_potential else 2)
# in the scripts, the partials are computed up to a factor of gm
partials_scale = gm

# Computing the terms of the normal equations
# If sim_msr_vec is not None, it will be copied to msr_vec (and used to compute y),
# otherwise msr_vec and y are computed from the Stokes coefficients
N_mat, y, msr_vec = scripts.compute_normal_equations(
    cnm_mat_est, # coefficients used to simulate measurements (if sim_msr_vec is None)
    sph_coords, # measurements locations
    partials_func, # function for the partials computation
    r_0=r_0,
    batch_size=1000, # With 1000, up to 10GB RAM for n_max_est=100 and 9e4 msr
    msr_noise=msr_noise,
    partials_scale=partials_scale,
    raw_msr_vec=sim_msr_vec, # synthetic measurements (if None they will be computed from cnm_mat_est)
    perturb_msr=perturb_msr,  # contaminate synthetic measurements with Gaussian noise
    rng=rng,
)
print("Took {:.2f} s".format(time() - ts))

Computing Normal equations...


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 24.28it/s]

Took 0.23 s


### Least-squares solution

In [6]:
cnm_vec_sim = cnm_mat_sim[*cnm_idx_sim.T]  # ground-truth vector of Stokes coefficients

# solution and covariance for the estimated parameters
# if compute_covariance is False, cov will be None 
cnm_vec_est, cov = scripts.solve_normal_equations(
    N_mat, y, compute_covariance=compute_covariance
)

if compute_covariance:
    cnm_vec_sigma = np.sqrt(np.diagonal(cov))  # formal errors
    corr_mat = cov / np.outer(cnm_vec_sigma, cnm_vec_sigma)  # correlations matrix

# difference between estimated and ground-truth Stokes coefficients
n_shared_coeffs = min(cnm_vec_sim.shape[0], cnm_vec_est.shape[0])
cnm_true_errors = cnm_vec_est[:n_shared_coeffs] - cnm_vec_sim[:n_shared_coeffs]

Solving via SVD...
Took 0.03 s


### Plotting

In [7]:
val = msr_vec.reshape(-1, msr_dim)
file_path = "spice/data/bennu/bennu_mesh.pkl"
with open(file_path, "rb") as f:
    faces, verts = pk.load(f)
    
# 3D trajectory with measurement points
fig_trj = scripts.plot_traj(state_mat, val, val_lbl=val_lbl, scatter_size=1)
body_mesh_plot = go.Mesh3d(
    x=verts[:, 0],
    y=verts[:, 1],
    z=verts[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    color="gray",
    opacity=0.6,
    showlegend=False,
    name="Body",
    lighting=dict(
        ambient=0.6,
        diffuse=0.9,
        roughness=0.9,
        specular=0.1,
        fresnel=0.2,
    ),
)
fig_trj.add_trace(body_mesh_plot)
fig_out_path = os.path.join(out_path, "sim_msr_plot_3D.html")
fig_trj.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=1000, height=500)

In [8]:
# 2D plot of trajectory and measurements (in spherical coordinates)
plot_coords = sph_coords[:, [2, 1]]
plot_coords[:,1] = 90*deg- plot_coords[:,1]
fig_grav_2d = scripts.plot_val_2d(
    plot_coords/deg, val, val_lbl=val_lbl
)
fig_out_path = os.path.join(out_path, "sim_msr_plot_2D.html")
fig_grav_2d.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=800, height=500)

In [9]:
# Spectra of Sokes coefficients and their errors
cnm_spectrum_sim = np.array(
    [
        (n, np.linalg.norm(cnm_vec_sim[cnm_idx_sim[:, 1] == n]) / np.sqrt(2 * n + 1))
        for n in range(n_max_sim + 1)
    ]
)
cnm_spectrum_est = np.array(
    [
        (n, np.linalg.norm(cnm_vec_est[cnm_idx_est[:, 1] == n]) / np.sqrt(2 * n + 1))
        for n in range(n_max_est + 1)
    ]
)

# Line plots of the Cnm spectra
fig_spectrum = go.Figure()
fig_spectrum.add_traces(
    go.Scatter(
        x=cnm_spectrum_sim[2:, 0],
        y=cnm_spectrum_sim[2:, 1],
        mode="lines",
        line=dict(color="darkgrey", width=2),
        showlegend=True,
        name="Ground-truth",
    )
)
fig_spectrum.add_traces(
    go.Scatter(
        x=cnm_spectrum_est[2:, 0],
        y=cnm_spectrum_est[2:, 1],
        mode="lines",
        line=dict(color="royalblue", width=2),
        showlegend=True,
        name="Estimated",
    )
)
if compute_covariance:
    cnm_spectrum_sigma = np.array(
        [
            (
                n,
                np.linalg.norm(cnm_vec_sigma[cnm_idx_est[:, 1] == n])
                / np.sqrt(2 * n + 1),
            )
            for n in range(n_max_est + 1)
        ]
    )
    fig_spectrum.add_traces(
        go.Scatter(
            x=cnm_spectrum_sigma[2:, 0],
            y=cnm_spectrum_sigma[2:, 1],
            mode="lines",
            line=dict(color="royalblue", dash="dash", width=2),
            showlegend=True,
            name="Uncertainty",
        )
    )
fig_spectrum.update_layout(
    xaxis_title=r"$l_{\max}$", yaxis_title="Power spectrum", yaxis_type="log"
)
fig_out_path = os.path.join(out_path, "spectrum.html")
fig_spectrum.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=800, height=500)

#### Error statistics
Here we evaluate the synthetic and the estimated Stokes coefficients over a same GLQ grid, computed for degree `n_max` (maximum of `n_max_sim` and `n_max_est`) and at a radial distance of $1.1R_0$

In [10]:
print("Computing errors on GLQ grid...")
ts = time()

# constructing matrix from vector of estimated coefficients
cnm_mat_est = cnm_mat_est.astype(np.double)
cnm_mat_est[*cnm_idx_est.T] = cnm_vec_est

# cartesian (state_grid) and spherical(rtp_grid) coordinates of the GLQ grid points
state_grid, rtp_grid = scripts.points_from_grid(n_max, r=(r_0 * 1.1), use_GLQ_grid=True)

# forward gravity computation for ground-truth coefficients
# The function outputs are up to a factor of gm, so we multiply them by gm
pot_sim_grid, acc_sim_grid = [
    el * gm for el in scripts.compute_pot_acc(cnm_mat_sim, r_0, rtp_grid)
]
# forward gravity computation for estimated coefficients
pot_est_grid, acc_est_grid = [
    el * gm for el in scripts.compute_pot_acc(cnm_mat_est, r_0, rtp_grid)
]
print("Took {:.2f} s".format(time() - ts))

Computing errors on GLQ grid...


100%|█████████████████████████████████████████| 11/11 [00:00<00:00, 1059.29it/s]

Took 0.06 s


In [11]:
# difference
err_grid = pot_sim_grid - pot_est_grid if use_potential else acc_sim_grid - acc_est_grid
err_grid = err_grid.reshape(-1, msr_dim)

rel_err = err_grid / ((pot_sim_grid if use_potential else acc_sim_grid) + 1e-20)

# Mean, median, RMS, and SNR of the errors
err_stats = [
    np.mean(err_grid),
    np.median(err_grid),
    np.linalg.norm(err_grid) / np.sqrt(err_grid.size),
]
err_stats.append(np.log10(np.abs(np.mean(1 / (rel_err + 1e-20)))))
print("Error stats: ")
print("Mean: {:.5g} - Median: {:.5g} -  RMS: {:.5g} - SNR (dB):{:.5g}".format(*err_stats))

# 3D plot of the errors
fig_diff =scripts.plot_traj(state_grid, err_grid, val_lbl="Error " + val_lbl)
fig_out_path = os.path.join(out_path, "true_errors_grid.html")
fig_diff.write_html(
        fig_out_path,
        include_mathjax="cdn",
        include_plotlyjs="cdn",
        config=dict({"scrollZoom": True}),
    )
IFrame(src=fig_out_path, width=1000, height=500)

Error stats: 
Mean: 6.2236e-10 - Median: -2.6016e-10 -  RMS: 1.2019e-07 - SNR (dB):1.4596


### Correlation plot

In [12]:
if compute_covariance and n_max<=10:
    # Heatmap of the correlation matrix
    _customdata = np.array(
        [
            "{}_{:d},{:d} - {}_{:d},{:d}".format(
                "C" if el_1[0] == 0 else "S",
                el_1[1],
                el_1[2],
                "C" if el_2[0] == 0 else "S",
                el_2[1],
                el_2[2],
            )
            for el_1 in cnm_idx_est
            for el_2 in cnm_idx_est
        ]
    ).reshape(corr_mat.shape)
    _hovertemplate="<b>%{customdata}</b><br>%{z:.3f}"
    fig_corr = go.Figure(
        data=go.Heatmap(
            z=corr_mat,
            customdata=_customdata,
            hovertemplate=_hovertemplate,
            colorscale="RdBu_r",
        )
    )
    fig_out_path = os.path.join(out_path, "correlations.html")
    fig_corr.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
    IFrame(src=fig_out_path, width=500, height=500)

### Storing results

In [13]:
# Saving Stokes coefficients in the SHADR format. Can be read via scripts.read_shadr or in SHTOOLS via sh.SHGravCoeffs.from_file
cnm_gt_file = "cnm_ground_truth"
scripts.write_SHADR(
    os.path.join(out_path, cnm_gt_file + ".txt"),
    cnm_vec_sim,
    cnm_map_sim,
    gm=gm,
    r_0=r_0,
)

cnm_sol_file = "cnm_estimated"
scripts.write_SHADR(
    os.path.join(out_path, cnm_sol_file + ".txt"),
    cnm_vec_est,
    cnm_map_est,
    gm=gm,
    r_0=r_0,
)

# Saving measurements evaluated on a spherical grid
grid_gt_file = "grid_ground_truth"
scripts.save_msr_grid(
    os.path.join(out_path, grid_gt_file + ".pkl"),
    rtp_grid,
    pot_sim_grid if use_potential else acc_sim_grid,
)
grid_est_file = "grid_estimated"
scripts.save_msr_grid(
    os.path.join(out_path, grid_est_file + ".pkl"),
    rtp_grid,
    pot_est_grid if use_potential else acc_est_grid,
)